In [7]:
import os
from dotenv import load_dotenv

load_dotenv()
tapo_username = os.getenv("TAPO_USERNAME")
tapo_password = os.getenv("TAPO_PASSWORD")
pushover_user_group = os.getenv("PUSHOVER_USER_GROUP_WOERIS")

In [5]:
pushover_user_group

'gh9maxhrd44jxy8qx7ka1jc2cfiykg'

In [6]:
from utils import send_pushover_notification
send_pushover_notification(pushover_user_group, "Test")

{"status":1,"request":"18aa3dbb-38d1-4067-bf78-08648d6dade9"}


In [4]:
import http.client, urllib
conn = http.client.HTTPSConnection("api.pushover.net:443")

pushover_api_token = os.getenv("PUSHOVER_TAPO_API_TOKEN")
conn.request("POST", "/1/messages.json",
  urllib.parse.urlencode({
    "token": pushover_api_token,
    "user": pushover_user_group,
    "message": "hello world",
  }), { "Content-type": "application/x-www-form-urlencoded" })
conn.getresponse()

In [8]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = os.environ.get("INFLUXDB_TOKEN")
org = "None"
url = "http://192.168.178.114:8088"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

In [9]:
bucket="test_bucket"

write_api = write_client.write_api(write_options=SYNCHRONOUS)
   
for value in range(5):
    point = (
        Point("measurement1")
        .tag("tagname1", "tagvalue1")
        .field("field1", value)
    )
    write_api.write(bucket=bucket, org="None", record=point)
    time.sleep(1) # separate points by 1 second

In [11]:
query_api = write_client.query_api()

query = """from(bucket: "test_bucket")
    |> range(start: -10m)
    |> filter(fn: (r) => r._measurement == "measurement1")"""
tables = query_api.query(query, org="None")

for table in tables:
    for record in table.records:
        print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 12, 13, 15, 18, 25, 376630, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2024, 12, 13, 15, 28, 25, 376630, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2024, 12, 13, 15, 26, 20, 935153, tzinfo=datetime.timezone.utc), '_value': 0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 12, 13, 15, 18, 25, 376630, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2024, 12, 13, 15, 28, 25, 376630, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2024, 12, 13, 15, 26, 23, 421039, tzinfo=datetime.timezone.utc), '_value': 1, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 12, 13, 15, 18, 25, 376630, tzinfo=datetime.timezone.utc), '_

In [12]:
query_api = write_client.query_api()

query = """from(bucket: "test_bucket")
    |> range(start: -10m)
    |> filter(fn: (r) => r._measurement == "measurement1")
    |> mean()"""
tables = query_api.query(query, org="None")

for table in tables:
    for record in table.records:
        print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 12, 13, 15, 19, 1, 732013, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2024, 12, 13, 15, 29, 1, 732013, tzinfo=datetime.timezone.utc), '_value': 2.0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}


In [13]:
tables

[<FluxTable: 8 columns, 1 records>]